### Import Data

In [23]:
import pandas as pd
import numpy as np

gdp = pd.read_csv('data/GDP.csv')
real_gdp_QoQ = pd.read_csv('data/real GDP QoQ.csv')
real_gdp_YoY = pd.read_csv('data/gdp real YoY.csv')
inflation = pd.read_csv('data/inflation.csv')
rates = pd.read_csv('data/rates.csv')
sp500 = pd.read_csv('data/sp500_data.csv')
unemployment = pd.read_csv('data/Unemployment.csv')
debt_22 = pd.read_csv('data/debt_22.csv')
debt_23 = pd.read_csv('data/debt_23.csv')
m2 = pd.read_csv('data/m2.csv')

### Nulls? Bring them to me!

In [24]:
dict_dataframes = {
    'gdp': gdp,
    'real_gdp_QoQ': real_gdp_QoQ,
    'real_gdp_YoY': real_gdp_YoY,
    'inflation': inflation,
    'rates': rates,
    'sp500': sp500,
    'unemployment': unemployment,
    'debt_22': debt_22,
    'debt_23': debt_23,
    'm2' : m2
}

# If missing values, replace with NaN
for df in dict_dataframes.values():
    df.replace([" ", ""], np.nan, inplace=True)

separator = "~" * 50 # printing separator
for name, df in dict_dataframes.items():
    null_counts = df.isnull().sum()
    if null_counts.sum() > 0:
        print(f"Attention! Nulls in -> {name}", "\n")
        print(null_counts,"\n\n", separator,"\n")

Attention! Nulls in -> unemployment 

Year    0
Jan     0
Feb     0
Mar     0
Apr     0
May     0
Jun     0
Jul     0
Aug     1
Sep     1
Oct     1
Nov     1
Dec     1
dtype: int64 

 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 

Attention! Nulls in -> debt_23 

Record Date                         0
Debt Held by the Public          2958
Intragovernmental Holdings       2958
Total Public Debt Outstanding       0
Source Line Number                  0
Fiscal Year                         0
Fiscal Quarter Number               0
Calendar Year                       0
Calendar Quarter Number             0
Calendar Month Number               0
Calendar Day Number                 0
dtype: int64 

 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 



#### Same type and name for Date columns

In [25]:
# See columns to renmae DATE (exception -> unemployment)
for name, df in dict_dataframes.items():
    print(f"{name} -> {list(df.columns)}", "\n")

# Same dates format confirmed (exception -> unemployment)
sp500.rename(columns={'Date': 'DATE'}, inplace=True)
debt_22.rename(columns={'Record Date': 'DATE'}, inplace=True)
debt_23.rename(columns={'Record Date': 'DATE'}, inplace=True)

# Change DATE to pd.to_datetime
for name, df in dict_dataframes.items():
    if name != 'unemployment':
        df['DATE'] = pd.to_datetime(df['DATE'])

gdp -> ['DATE', 'GDP'] 

real_gdp_QoQ -> ['DATE', 'A191RL1Q225SBEA'] 

real_gdp_YoY -> ['DATE', 'GDPC1_PC1'] 

inflation -> ['DATE', 'ALL', 'LESS_FOOD_ENERGY'] 

rates -> ['DATE', 'DFF'] 

sp500 -> ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'] 

unemployment -> ['Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'] 

debt_22 -> ['Record Date', 'Debt Outstanding Amount', 'Source Line Number', 'Fiscal Year', 'Fiscal Quarter Number', 'Calendar Year', 'Calendar Quarter Number', 'Calendar Month Number', 'Calendar Day Number'] 

debt_23 -> ['Record Date', 'Debt Held by the Public', 'Intragovernmental Holdings', 'Total Public Debt Outstanding', 'Source Line Number', 'Fiscal Year', 'Fiscal Quarter Number', 'Calendar Year', 'Calendar Quarter Number', 'Calendar Month Number', 'Calendar Day Number'] 

m2 -> ['DATE', 'M2SL'] 



#### Same Dates period

In [26]:
# We know that some data are more recent than others
# but we want to work in a "recent" period
for name, df in dict_dataframes.items():
    if name == 'unemployment':
        continue
    if df['DATE'].min() < pd.to_datetime('1970-01-01'):
        print(f"Dates before 1970-01-01 in -> {name}")

gdp = gdp[gdp['DATE'] >= pd.to_datetime('1970-01-01')]
debt_22 = debt_22[debt_22['DATE'] >= pd.to_datetime('1970-01-01')]
print("\nCorrected, dates now start from 1970.")

Dates before 1970-01-01 in -> gdp
Dates before 1970-01-01 in -> debt_22

Corrected, dates now start from 1970.


#### Function to test Dates:

In [27]:
import datetime

print("As the dates have different periods, they will be checked in each case.")

# Check if all DFs are in the range of expected dates
def test_dates(df, start:str, end:str, freq:str, help=False, missing_dates_list=False):
    """
    Test if all values are in the range of expected dates.
    """
    if help:
        print("See 'freq' values in: https://pandas.pydata.org/docs/user_guide/timeseries.html#offset-aliases\n")
    start_date = datetime.date.fromisoformat(start)
    end_date = datetime.date.fromisoformat(end)
    range_date = pd.date_range(start_date, end_date, freq=freq)
    may_be_empty = range_date[~range_date.isin(df['DATE'])].empty
    if may_be_empty:
        print(f'All good, not missing dates!')
    else:
        print(f'Attention! Missing dates!!!!')
        if missing_dates_list:
            return [date.date().strftime('%Y-%m-%d') for date in range_date[~range_date.isin(df['DATE'])]]
        print(f'Missing dates: {range_date[~range_date.isin(df["DATE"])]}')

As the dates have different periods, they will be checked in each case.


#### Function to calculate Percentage Change:

Given the difference in scales, calculating percentages of change could be very useful in future comparisons.

- For percentage change in a short period (daily-weekly), we will use '**PERC_**' prefix
- In other cases, we will use '**_YoY**', '**_QoQ**' or '**_MoM**' suffixes

Calculate on the following data:
- Quarterly -> GDPs
- Monthly -> M2
- Daily-weekly -> SP
- Yearly-Monthy-Daily -> Debt

In [28]:
def change_rate(df,column, new_column_name, time_gap):
    """
    Return same df with the new column.
    The percentage of change is calculated according to x 'time_gap'
    or jumps on data (rows).
    """
    df = df.copy()
    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html
    df[new_column_name] = df[column].pct_change(periods=time_gap) * 100
    return df

### PREPROCESSING GDPs

In [29]:
test_dates(gdp, '1970-01-01', '2023-04-01', 'QS', help=True)
test_dates(real_gdp_QoQ, '1970-01-01', '2023-04-01', 'QS')
test_dates(real_gdp_YoY, '1970-01-01', '2023-04-01', 'QS')

See 'freq' values in: https://pandas.pydata.org/docs/user_guide/timeseries.html#offset-aliases

All good, not missing dates!
All good, not missing dates!
All good, not missing dates!


In [30]:
# Merge DFs
gdp_us = pd.merge(gdp, real_gdp_QoQ, on='DATE', how='left')
gdp_us = pd.merge(gdp_us, real_gdp_YoY, on='DATE', how='left')
gdp_us.rename(columns={'A191RL1Q225SBEA': 'REAL_GDP_QoQ', 'GDPC1_PC1': 'REAL_GDP_YoY'}, inplace=True)
gdp_us.head()

,DATE,GDP,REAL_GDP_QoQ,REAL_GDP_YoY
0,1970-01-01,1051.200,-0.6,0.32493
1,1970-04-01,1067.375,0.6,0.16284
2,1970-07-01,1086.059,3.7,0.42258
3,1970-10-01,1088.608,-4.2,-0.16670
4,1971-01-01,1135.156,11.3,2.69724


In [31]:
gdp_us = change_rate(gdp_us, 'GDP', 'GDP_QoQ', 1)
gdp_us = change_rate(gdp_us, 'GDP', 'GDP_YoY', 4)
gdp_us.head()

,DATE,GDP,REAL_GDP_QoQ,REAL_GDP_YoY,GDP_QoQ,GDP_YoY
0,1970-01-01,1051.200,-0.6,0.32493,NaN,NaN
1,1970-04-01,1067.375,0.6,0.16284,1.538718,NaN
2,1970-07-01,1086.059,3.7,0.42258,1.750463,NaN
3,1970-10-01,1088.608,-4.2,-0.16670,0.234702,NaN
4,1971-01-01,1135.156,11.3,2.69724,4.275919,7.986682


### PREPROCESSING M2

*As you can see [here](https://fred.stlouisfed.org/series/M2SL), **curiously** the M2 metric changes measurement in March 2020. To avoid errors, it's split into 2 periods.*

In [32]:
# MS = Month Start/ calendar month begin
test_dates(m2, "1970-01-01", "2023-06-01", freq="MS")

# Splited Dfs
new_M2 = m2[m2['DATE'] >= pd.to_datetime('2020-03-01')].copy()
new_M2.rename(columns={'M2SL': 'NEW_M2'}, inplace=True)
m2_copy = m2[m2['DATE'] < pd.to_datetime('2020-03-01')]
m2_copy.tail()
#New_M2.head()

All good, not missing dates!


,DATE,M2SL
597,2019-10-01,15140.8
598,2019-11-01,15242.2
599,2019-12-01,15320.7
600,2020-01-01,15396.0
601,2020-02-01,15450.3


In [33]:
# Percentage change -> Monthly
m2_copy = change_rate(m2_copy, 'M2SL', 'M2_MoM', 1)
new_M2 = change_rate(new_M2, 'NEW_M2', 'NEW_M2_MoM', 1)
new_M2.head()

,DATE,NEW_M2,NEW_M2_MoM
602,2020-03-01,15978.7,NaN
603,2020-04-01,16997.6,6.376614
604,2020-05-01,17851.1,5.021297
605,2020-06-01,18131.7,1.571892
606,2020-07-01,18286.3,0.852650


### PREPROCESSING DEBT DFs

In [34]:
# OBJECTIVE: GET DATE (TEST IT) AND DEBT OUTSTANDING AMOUNT
# Note that we don't care about the 'fiscal' columns

print("Since some columns are redundant ('Date' provides it)")
print("Can the following columns be removed?\n")

# Is the value of "Fiscal Year" repeated?
print('Fiscal Year ->',debt_22['Fiscal Year'].
                  equals(debt_22['Calendar Year']))
# Same question...
print("Calendar Year ->", debt_22['DATE'].dt.year.astype(int).
                    equals(debt_22['Calendar Year'].astype(int)))

print("Calendar Month Number ->", debt_22['DATE'].dt.month.astype(int).
                        equals(debt_22['Calendar Month Number'].astype(int)))

print("Calendar Day Number ->", debt_22['DATE'].dt.day.astype(int).
                      equals(debt_22['Calendar Day Number'].astype(int)))

# As the dates have different periods, 1 per year is expected.
print("\nDates expected?", len(debt_22['DATE'].dt.year.unique()) == len(debt_22['DATE']))

debt_22_copy = debt_22.copy()
debt_22_copy.rename(columns={'Debt Outstanding Amount': 'DEBT_AMOUNT'}, inplace=True)
# Drop all except date and debt
debt_22_copy = debt_22_copy[['DATE', 'DEBT_AMOUNT']]
debt_22_copy.head()

Since some columns are redundant ('Date' provides it)
Can the following columns be removed?

Fiscal Year -> True
Calendar Year -> True
Calendar Month Number -> True
Calendar Day Number -> True

Dates expected? True


,DATE,DEBT_AMOUNT
0,2022-09-30,3.092891e+13
1,2021-09-30,2.842892e+13
2,2020-09-30,2.694539e+13
3,2019-09-30,2.271940e+13
4,2018-09-30,2.151606e+13


In [35]:
print("Same questions and context on similar df:\n")

print('Can "Calendar Year" be removed?',
      debt_23['DATE'].dt.year.astype(int).
        equals(debt_23['Calendar Year'].astype(int)))

print('Can "Calendar Month Number" be removed?',
      debt_23['DATE'].dt.month.astype(int).
        equals(debt_23['Calendar Month Number'].astype(int)))

print('Can "Calendar Day Number" be removed?',
      debt_23['DATE'].dt.day.astype(int).
        equals(debt_23['Calendar Day Number'].astype(int)))

debt_23_copy = debt_23.copy()
debt_23_copy['DEBT_AMOUNT'] = debt_23['Total Public Debt Outstanding']
debt_23_copy = debt_23_copy[['DATE', 'DEBT_AMOUNT']]

# La suma de los valores nulos en cada columna es == 0 ?
print("\nDataFrame without nulls -> ",
      debt_23_copy.isnull().sum().sum() == 0)

debt_23_copy.head(3)

Same questions and context on similar df:

Can "Calendar Year" be removed? True
Can "Calendar Month Number" be removed? True
Can "Calendar Day Number" be removed? True

DataFrame without nulls ->  True


,DATE,DEBT_AMOUNT
0,2023-08-17,3.270369e+13
1,2023-08-16,3.266158e+13
2,2023-08-15,3.267994e+13


##### Union of DEBT DFs

In [36]:
# Check for repeated dates and values:

# Date to string for efficiency
debt_22_copy['DATE_str'] = debt_22_copy['DATE'].dt.strftime('%Y-%m-%d')
debt_23_copy['DATE_str'] = debt_23_copy['DATE'].dt.strftime('%Y-%m-%d')

in_both_23_22 = len(debt_22_copy[debt_22_copy['DATE_str'].isin(debt_23_copy['DATE_str'].values)])
print(f"There are {in_both_23_22} overlapping dates.")

# Test values 
for index_22, row_22 in debt_22_copy.iterrows():
    date_str = row_22['DATE_str']
    debt_amount_22 = row_22['DEBT_AMOUNT']
    
    if date_str in debt_23_copy['DATE_str'].values:
        debt_amount_23 = debt_23_copy[debt_23_copy['DATE_str'] == date_str]['DEBT_AMOUNT'].values[0]
        
        if debt_amount_22 != debt_amount_23:
            print(f"DATE: {date_str}, DEBT_AMOUNT_22: {debt_amount_22}, DEBT_AMOUNT_23: {debt_amount_23}")
            print(f"Given a single error for {debt_amount_22 - debt_amount_23}$")

# Prints to show the error
print("In a scale of trillions, it is taken as anecdotal.")
print("However, the average is taken as a correction.\n")
# Average of the two values
new_value = (5656270901615.43 + 5656270901633.43) / 2

debt_22_copy.drop(columns=['DATE_str'], inplace=True)
debt_23_copy.drop(columns=['DATE_str'], inplace=True)

# Substitute the value
debt_22_copy.loc[debt_22_copy['DATE'] == '1999-09-30', 'DEBT_AMOUNT'] = new_value
debt_23_copy.loc[debt_23_copy['DATE'] == '1999-09-30', 'DEBT_AMOUNT'] = new_value

There are 23 overlapping dates.
DATE: 1999-09-30, DEBT_AMOUNT_22: 5656270901615.43, DEBT_AMOUNT_23: 5656270901633.43
Given a single error for -18.0$
In a scale of trillions, it is taken as anecdotal.
However, the average is taken as a correction.



In [37]:
# Merge dfs
debt = pd.merge(debt_22_copy, debt_23_copy, on='DATE', how='outer', suffixes=('_22', '_23'))
# Make a single column (DEBT_AMOUNT)
debt['DEBT_AMOUNT'] = debt['DEBT_AMOUNT_22'].combine_first(debt['DEBT_AMOUNT_23'])
debt.drop(columns=['DEBT_AMOUNT_22', 'DEBT_AMOUNT_23'], inplace=True)
debt['DATE'] = pd.to_datetime(debt['DATE'])
debt.sort_values(by=['DATE'], inplace=True)
debt.reset_index(drop=True, inplace=True)
debt.head()

,DATE,DEBT_AMOUNT
0,1970-06-30,3.709187e+11
1,1971-06-30,3.981297e+11
2,1972-06-30,4.272605e+11
3,1973-06-30,4.581416e+11
4,1974-06-30,4.750598e+11


If you remember, the data do not follow stable periods (years, days, etc.).

That is why **YoY** will be calculated with the latest annual data in each case.

In [38]:
# Get last date avaible per year
last_date_per_year = debt.groupby(debt['DATE'].dt.year)['DATE'].transform('max')
result = debt[debt['DATE'] == last_date_per_year]

result = change_rate(result, 'DEBT_AMOUNT', 'DEBT_AMOUNT_YoY', 1)
# 2023 is not a full year, so it is removed
result = result[:-1]

# Merge result with debt
debt = pd.merge(debt, result[['DATE', 'DEBT_AMOUNT_YoY']], on='DATE', how='left')
debt.head()

,DATE,DEBT_AMOUNT,DEBT_AMOUNT_YoY
0,1970-06-30,3.709187e+11,NaN
1,1971-06-30,3.981297e+11,7.336119
2,1972-06-30,4.272605e+11,7.316890
3,1973-06-30,4.581416e+11,7.227709
4,1974-06-30,4.750598e+11,3.692791


### PREPROCESSING S&P 500

In [39]:
# In this case we only care about Date, Close and Volume
sp500 = sp500[['DATE', 'Close', 'Volume']].copy()
sp500.rename(columns={'Close': 'SP', 'Volume': 'SP_VOL'}, inplace=True)
sp500.columns = sp500.columns.str.upper()
print(sp500.head(3), "\n")
print("Repeated dates?", sp500['DATE'].duplicated().any())
sp500.head(3)

        DATE         SP    SP_VOL
0 1970-01-02  93.000000   8050000
1 1970-01-05  93.459999  11490000
2 1970-01-06  92.820000  11460000 

Repeated dates? False


,DATE,SP,SP_VOL
0,1970-01-02,93.000000,8050000
1,1970-01-05,93.459999,11490000
2,1970-01-06,92.820000,11460000


In this case, the market closes on weekends, some holidays and special events, so checking dates is a very fun exercise!

In [40]:
import pandas as pd
from pandas.tseries.holiday import AbstractHolidayCalendar, USFederalHolidayCalendar, Holiday, nearest_workday
from pandas.tseries.holiday import USMemorialDay, USLaborDay, USThanksgivingDay, GoodFriday, USPresidentsDay, USMartinLutherKingJr

# As there are some gaps on date, let's see which are expected
testing = test_dates(sp500, '1970-01-02', '2023-08-14', freq='D', missing_dates_list=True)
print(f"\n{len(testing)} dates are missing.", "\n", testing[:5], "...")
# Remove weekends
filtered_testing = [dates for dates in testing
                    if datetime.datetime.strptime(dates, '%Y-%m-%d').
                    date().weekday() not in [5, 6]]

# Create a calendar
calendar = USFederalHolidayCalendar()

# Add custom holidays
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#holidays-holiday-calendars
class CustomHolidayCalendar(AbstractHolidayCalendar):
    rules = [
        # Where nearest_workday moves Saturday to Friday and Sunday to Monday
        Holiday('New Years Day', month=1, day=1, observance=nearest_workday),
        USMemorialDay,
        Holiday('July 4th', month=7, day=4, observance=nearest_workday),
        USLaborDay,
        USThanksgivingDay,
        Holiday('Christmas', month=12, day=25, observance=nearest_workday),
        USPresidentsDay,
        GoodFriday,
        USMartinLutherKingJr,
    ]

calendar = CustomHolidayCalendar()
holidays = calendar.holidays(start='1970-01-01', end='2023-12-31')
date_list = [pd.to_datetime(date) for date in filtered_testing]

# Check how many are missing
count = 0

for date in date_list:
    if date not in holidays:
        count += 1
        #print(f"{date}: Not expected to be a holiday.")

print(f"\n{count} MISSING DATES IN MORE THAN 50 YEARS.")
print("You can check which events occurred using the commented print, \
but this result is enough to calculate the percentage.")

Attention! Missing dates!!!!

6060 dates are missing. 
 ['1970-01-03', '1970-01-04', '1970-01-10', '1970-01-11', '1970-01-17'] ...

20 MISSING DATES IN MORE THAN 50 YEARS.
You can check which events occurred using the commented print, but this result is enough to calculate the percentage.


In [41]:
sp500 = change_rate(sp500, 'SP', 'PERC_SP', 1)
sp500 = change_rate(sp500, 'SP_VOL', 'PERC_SP_VOL', 1)
sp500.head()

,DATE,SP,SP_VOL,PERC_SP,PERC_SP_VOL
0,1970-01-02,93.000000,8050000,NaN,NaN
1,1970-01-05,93.459999,11490000,0.494623,42.732919
2,1970-01-06,92.820000,11460000,-0.684784,-0.261097
3,1970-01-07,92.629997,10010000,-0.204700,-12.652705
4,1970-01-08,92.680000,10670000,0.053981,6.593407


### INFLATION or CPI - CLOSE LOOK

In [42]:
test_dates(inflation, '1970-01-01', '2023-07-01', freq='MS')
inflation.head()

All good, not missing dates!


,DATE,ALL,LESS_FOOD_ENERGY
0,1970-01-01,6.16246,6.16622
1,1970-02-01,6.42458,5.85106
2,1970-03-01,6.09418,6.08466
3,1970-04-01,6.06061,6.03675
4,1970-05-01,6.04396,6.29921


### Preprocessing UNEMPLOYMENT data

In [43]:
unemployment.head(5)

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1948,3.4,3.8,4.0,3.9,3.5,3.6,3.6,3.9,3.8,3.7,3.8,4.0
1,1949,4.3,4.7,5.0,5.3,6.1,6.2,6.7,6.8,6.6,7.9,6.4,6.6
2,1950,6.5,6.4,6.3,5.8,5.5,5.4,5.0,4.5,4.4,4.2,4.2,4.3
3,1951,3.7,3.4,3.4,3.1,3.0,3.2,3.1,3.1,3.3,3.5,3.5,3.1
4,1952,3.2,3.1,2.9,2.9,3.0,3.0,3.2,3.4,3.1,3.0,2.8,2.7


In [44]:
unemployment.columns = unemployment.columns.str.upper()
# As we did not transform the dates before:
unemployment_seventies = unemployment[unemployment['YEAR'] >= 1970]
# Transform DF to earlier formats (yyyy-mm-dd):

# FIRST:  [YYYY] ([M(string)] i.e. [Jan, Feb, ...])
unemployment_melted = unemployment_seventies.melt(id_vars=['YEAR'], var_name='MONTH', value_name='UNEMPLOYMENT_RATE')

# At this point we have something like:
# 	YEAR	MONTH	UNEMPLOYMENT_RATE
# 0	1970	JAN	    3.9
# 1	1971	JAN	    5.9

# Now we add the day (always 1)
unemployment_melted['DATE'] = pd.to_datetime(
    unemployment_melted['YEAR'].astype(str) + '-' + 
    unemployment_melted['MONTH'] + '-01',
    format='%Y-%b-%d'  # Where %b equals month, format:(Jan, ...)
)
# # Recommended source of formats:
# # https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes

unemployment_melted = unemployment_melted.sort_values(by='DATE')
unemployment_melted = unemployment_melted[['DATE', 'UNEMPLOYMENT_RATE']]
unemployment_melted.reset_index(drop=True, inplace=True)
unemployment_melted.tail(10)

,DATE,UNEMPLOYMENT_RATE
638,2023-03-01,3.5
639,2023-04-01,3.4
640,2023-05-01,3.7
641,2023-06-01,3.6
642,2023-07-01,3.5
643,2023-08-01,NaN
644,2023-09-01,NaN
645,2023-10-01,NaN
646,2023-11-01,NaN
647,2023-12-01,NaN


### FEDERAL RATES - CLOSE LOOK

In [45]:
rates.rename(columns={'DFF': 'RATES'}, inplace=True)
test_dates(rates, '1970-01-01', '2023-08-15', freq='D')
rates.head()

All good, not missing dates!


,DATE,RATES
0,1970-01-01,5.00
1,1970-01-02,9.63
2,1970-01-03,9.63
3,1970-01-04,9.63
4,1970-01-05,9.75


### Make them one!

In [46]:
all_dfs = {
    "gdp_us" : gdp_us,
    "m2_copy" : m2_copy,
    "new_M2" : new_M2,
    "debt" : debt,
    "sp500" : sp500,
    "inflation" : inflation,
    "unemployment_melted" : unemployment_melted,
    "rates" : rates
}

In [47]:
# Maximum dates for each df
name_max_length = max([len(name) for name in all_dfs.keys()])

for df, name in zip(all_dfs.values(), all_dfs.keys()):
    print(f"{name.ljust(name_max_length)} \t\t {df['DATE'].max().date()}")

gdp_us              		 2023-04-01
m2_copy             		 2020-02-01
new_M2              		 2023-06-01
debt                		 2023-08-17
sp500               		 2023-08-14
inflation           		 2023-07-01
unemployment_melted 		 2023-12-01
rates               		 2023-08-15


In [48]:
us_data = pd.concat([df.set_index('DATE') for df in all_dfs.values()], axis=1, join='outer')
us_data.reset_index(inplace=True)
us_data

,DATE,GDP,REAL_GDP_QoQ,REAL_GDP_YoY,GDP_QoQ,GDP_YoY,M2SL,M2_MoM,NEW_M2,NEW_M2_MoM,DEBT_AMOUNT,DEBT_AMOUNT_YoY,SP,SP_VOL,PERC_SP,PERC_SP_VOL,ALL,LESS_FOOD_ENERGY,UNEMPLOYMENT_RATE,RATES
0,1970-01-01,1051.2,-0.6,0.32493,NaN,NaN,589.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.16246,6.16622,3.9,5.00
1,1970-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.000000,8050000.0,NaN,NaN,NaN,NaN,NaN,9.63
2,1970-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.63
3,1970-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.63
4,1970-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.459999,11490000.0,0.494623,42.732919,NaN,NaN,NaN,9.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19586,2023-08-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.270369e+13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19587,2023-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19588,2023-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19589,2023-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# Since unemployment offers dates with null values (future),
# only 4 rows should appear with nulls (not date)
expected = [19587, 19588, 19589, 19590]

for i in range(len(us_data)):
    if us_data.iloc[i,1:].isnull().all():
        print("Row", i,
              "matches expectations->", i == expected.pop(0))

us_data.drop(us_data.tail(4).index, inplace=True)
us_data.tail()

Row 19587 matches expectations-> True
Row 19588 matches expectations-> True
Row 19589 matches expectations-> True
Row 19590 matches expectations-> True


,DATE,GDP,REAL_GDP_QoQ,REAL_GDP_YoY,GDP_QoQ,GDP_YoY,M2SL,M2_MoM,NEW_M2,NEW_M2_MoM,DEBT_AMOUNT,DEBT_AMOUNT_YoY,SP,SP_VOL,PERC_SP,PERC_SP_VOL,ALL,LESS_FOOD_ENERGY,UNEMPLOYMENT_RATE,RATES
19582,2023-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.33
19583,2023-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.266353e+13,NaN,4489.720215,3.896410e+09,0.575048,3.813188,NaN,NaN,NaN,5.33
19584,2023-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.267994e+13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.33
19585,2023-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.266158e+13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19586,2023-08-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.270369e+13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
# Rename
us_data.rename(columns={'M2SL': 'M2',
                        'ALL': 'INFLATION',
                        'LESS_FOOD_ENERGY': 'CORE_INFLATION'}, inplace=True)

# Reorder columns:
print(us_data.shape) # Check if I have forgotten any column
us_data = us_data[['DATE', 'GDP', 'GDP_QoQ', 'GDP_YoY', 'REAL_GDP_QoQ', 'REAL_GDP_YoY',
                    'SP', 'PERC_SP', 'SP_VOL', 'PERC_SP_VOL',
                    'M2', 'M2_MoM', 'NEW_M2', 'NEW_M2_MoM',
                    'DEBT_AMOUNT', 'DEBT_AMOUNT_YoY',
                    'INFLATION', 'CORE_INFLATION',
                    'UNEMPLOYMENT_RATE', 'RATES']]
print(us_data.shape) # Check 2.

(19587, 20)
(19587, 20)


### Save it!

In [52]:
# Save csv
us_data.to_csv('data/us.csv', index=False)
us_data.head()

,DATE,GDP,GDP_QoQ,GDP_YoY,REAL_GDP_QoQ,REAL_GDP_YoY,SP,PERC_SP,SP_VOL,PERC_SP_VOL,M2,M2_MoM,NEW_M2,NEW_M2_MoM,DEBT_AMOUNT,DEBT_AMOUNT_YoY,INFLATION,CORE_INFLATION,UNEMPLOYMENT_RATE,RATES
0,1970-01-01,1051.2,NaN,NaN,-0.6,0.32493,NaN,NaN,NaN,NaN,589.6,NaN,NaN,NaN,NaN,NaN,6.16246,6.16622,3.9,5.00
1,1970-01-02,NaN,NaN,NaN,NaN,NaN,93.000000,NaN,8050000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.63
2,1970-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.63
3,1970-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.63
4,1970-01-05,NaN,NaN,NaN,NaN,NaN,93.459999,0.494623,11490000.0,42.732919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.75
